In [68]:
import requests
import re
import warnings
import urllib.request

from webbot import Browser
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [69]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [70]:
common_words_list = pd.read_csv("Common_Articles_List.csv")
personality = pd.read_csv("Personality.csv", sep=',')
software = pd.read_csv("Programming_Languages.csv")

common = common_words_list["word"].to_list()
personality = [x.lower() for x in personality["Traits"]]
software = [x.lower() for x in software["languages"]]

In [71]:
def get_indeed_page_numbers(page):
    clock = False
    page_numbers_element = page.find_all("span", class_="pn", text=True)

    #First Loop To Give Us the 1st 5 Pages if available
    page_number_list = []
    for items in page_numbers_element:
        #print(items)
        page_number_list.append("https://www.indeed.com" + items.parent["href"])
    
    if len(page_numbers_element) == 4:
        
        loop_counter = 5
        
        while clock is False:
            last_page = page_number_list[-1]
            analysis_page = requests.get(last_page)
            soup = BeautifulSoup(analysis_page.content, "html.parser")
            new_link_list = soup.find_all("span", class_="pn", text=True)
            
            try:
                int(new_link_list[-1].text)
            except IndexError:
                clock = True
                
            additional_counter = int(new_link_list[-1].text) - loop_counter
            
            if additional_counter < 1: 
                clock = True
                
            else: 
                for items in new_link_list[(additional_counter * -1):]:
                    loop_counter = int(items.text)
                    page_number_list.append("https://www.indeed.com" + items.parent["href"])
                clock = False
        
        
    return page_number_list

In [72]:
def chrome_engine():
    options = Options()
    options.binary_location = "C:\Program Files\Google\Chrome\Application\chrome.exe"
    driver = webdriver.Chrome(ChromeDriverManager().install())

    #User Input and Driver Link
    driver.get('https://indeed.com/')
    job_title = driver.find_element_by_id('text-input-what')
    job_title.send_keys("Data Analyst")
    location = driver.find_element_by_id("text-input-where")
    location.send_keys("test")
    locate_clear_field = driver.find_element_by_class_name("icl-TextInputClearable-icon")
    locate_clear_field.click()
    fixed_location = location.send_keys("Washington, DC")
    submit_button = driver.find_element_by_class_name("yosegi-InlineWhatWhere-primaryButton")
    submit_button.send_keys(Keys.RETURN)
    current_page_url = driver.current_url
    
    return current_page_url

In [73]:
def indeed_page_web_scrapper(current_page_url):
    indeed_url = "https://www.indeed.com"

    analysis_page = requests.get(current_page_url)
    soup = BeautifulSoup(analysis_page.content, "html.parser")


    additional_page_links = get_indeed_page_numbers(soup)

    print(len(additional_page_links), " Potential Jobs Pages")




    job_links = []    
    job_listings = soup.find_all('a', class_="tapItem")
    counter = 0
    for job in job_listings:
        job_links.append(indeed_url + job["href"])


    for page_link in additional_page_links:
        web_page = requests.get(page_link)
        soup = BeautifulSoup(web_page.content, "html.parser")
        job_listings = soup.find_all('a', class_="tapItem")


        for job in job_listings:
            job_links.append(indeed_url + job["href"])


    print("Total Jobs Analyzed: ", len(job_links))
    return job_links

In [139]:
def job_description_cleaner(page_body_text):
    counter = 0
    unwanted_chars = [" ", ",", "/", "'\'", ".", "(", ")", ";", '"', "'", ":", "-", "_", 
                      "&", "[", "]", "*", "?", "#", "!", "%", "®", "“", '”', ":", "~"]

    additional_splitting = ["/", "(", ")", "%", "/", "."]
    final_list = []
    word_frequency = {}

    for items in page_body_text:
        text = re.sub("\n", " ", items)
        clean_list = text.split()

        for word in clean_list:
            split_words = []
            if word != "":

                if len(word) > 1:

                    cleaned = False
                    #print("Word Going Into Machine: ", word)
                    while cleaned is False:

                        if len(word) > 1 and word[len(word)-1] in unwanted_chars:
                            word = word[:-1]

                            if len(word) > 1 and word[0] in unwanted_chars:
                                word = word[1:]


                        if len(word) > 1 and word[0] in unwanted_chars:
                            word = word[1:]

                            if len(word) > 1 and word[len(word)-1] in unwanted_chars:
                                word = word[:-1]


                        if len(word) == 1:
                            if word in unwanted_chars:
                                #print("Word Deleted: ", word)
                                cleaned = True

                            else:
                                #print("Passed WORD = 1: ", word)
                                #print("   ")
                                final_list.append(word.lower().strip())
                                cleaned = True

                        elif word[0] not in unwanted_chars and word[len(word) - 1] not in unwanted_chars:
                            #print("Passed ELIF: ", word)
                            #print("   ")
                            if word[-2:] == "'s" or word[-2:] == "’s":
                                #print("Machine Processing S Condition: ", word)
                                word = word[:-2] + "s"
                                #print("Passed Condition: ", word)
                                #print("   ")

                            if "$" in word:
                                #print("Machine Processing Money Condition: ", word)
                                word = "$" + word.split("$", 1)[1]
                                #print("Passed Condition: ", word)
                                #print("   ")

                            for symbols in additional_splitting:
                                if symbols in word: 
                                    split_words = word.split(symbols)
                                    #print("Machine Processing Extra Symbols in Word Condition: ", word)
                                    #print("Passed Condition: ", split_words)
                                    #print("   ")

                            if len(split_words) == 0: 
                                final_list.append(word.lower().strip())
                                cleaned = True
                            else: 
                                for words in split_words:
                                    #print("Word Added Based On Extra Symbol Condition: ", words)
                                    #print("     ")
                                    final_list.append(words.lower().strip())
                                    cleaned = True

                        else: 
                            cleaned = False

                else: 
                    if word not in unwanted_chars:
                        #print("Passed SINGLE LETTER NOT IN CHARS: ", word)
                        #print("   ")
                        final_list.append(word.lower().strip())



    print("Done")
    print(len(final_list))
    return final_list

In [137]:
current_page_url = chrome_engine()
job_links = indeed_page_web_scrapper(current_page_url)

counter = 0
page_body_text = []

for job_page in job_links:
    loaded_page = requests.get(job_page)
    soup = BeautifulSoup(loaded_page.text, "html.parser")
        
    
    page_body = soup.find_all("div", class_="jobsearch-JobComponent-description")
        
    for elements in page_body:
        text = elements.get_text()
        
        page_body_text.append(text)
        counter += 1
        
print(len(job_links))
print("Total Job Descriptions Found: ", len(page_body_text))
print("DONE")



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\13017\.wdm\drivers\chromedriver\win32\100.0.4896.60]


64  Potential Jobs Pages
Total Jobs Analyzed:  775
775
Total Job Descriptions Found:  411
DONE


In [140]:
final_list = job_description_cleaner(page_body_text)

Done
240234


In [168]:
 for words in final_list:
        if words not in word_frequency:
            word_frequency[words] = 1
        else:
            word_frequency[words] += 1

len(word_frequency)

12820

In [169]:
for words in common: 
    if words in word_frequency.keys():
        del word_frequency[words]
    
sorted_word_frequency_list = sorted(word_frequency.items(), key=lambda x: x[1], reverse=True)


In [170]:
programming_languages = []
personality_traits = []
salary_expectations = []

for items in sorted_word_frequency_list: 
    if items[0] in software:
        programming_languages.append(items)
    
    if items[0] in personality:
        personality_traits.append(items)
        
    elif "$" in items[0]:
        if len(items[0]) == 3:
            money = items[0]
            points = items[1]
            money = money
            salary_expectations.append((money, points))
            continue
        salary_expectations.append(items)

In [171]:
programming_languages

[('sql', 2229),
 ('python', 1079),
 ('r', 855),
 ('sas', 511),
 ('c', 315),
 ('oracle', 265),
 ('aws', 220),
 ('javascript', 105),
 ('java', 66),
 ('xml', 65),
 ('ada', 41),
 ('html', 41),
 ('matlab', 35),
 ('mysql', 33),
 ('postgresql', 20),
 ('ruby', 15),
 ('css', 15),
 ('elasticsearch', 14),
 ('scala', 13),
 ('redis', 8),
 ('bash', 7),
 ('swift', 6),
 ('db2', 2)]

In [161]:
personality_traits

[('knowledge', 2717),
 ('complex', 1448),
 ('understanding', 1065),
 ('responsible', 845),
 ('critical', 723),
 ('innovative', 607),
 ('physical', 572),
 ('flexible', 552),
 ('competitive', 443),
 ('creative', 436),
 ('active', 424),
 ('focused', 357),
 ('firm', 348),
 ('religious', 340),
 ('dynamic', 321),
 ('dedicated', 282),
 ('passionate', 270),
 ('deep', 250),
 ('efficient', 226),
 ('political', 211),
 ('driving', 192),
 ('formal', 188),
 ('criminal', 173),
 ('generous', 167),
 ('independent', 164),
 ('challenging', 124),
 ('patient', 123),
 ('sensitive', 122),
 ('sharing', 119),
 ('solid', 119),
 ('practical', 113),
 ('thorough', 112),
 ('transparent', 109),
 ('exciting', 107),
 ('organized', 104),
 ('capable', 102),
 ('artificial', 100),
 ('logical', 98),
 ('confidential', 96),
 ('articulate', 86),
 ('determined', 85),
 ('objective', 81),
 ('adaptable', 81),
 ('curious', 80),
 ('directed', 77),
 ('secure', 75),
 ('progressive', 74),
 ('reliable', 72),
 ('soft', 70),
 ('dependent

In [128]:
salary_expectations

[('$77', 24),
 ('$80,000', 22),
 ('$85,000', 19),
 ('$45', 19),
 ('$90,000', 17),
 ('$65,000', 14),
 ('$69', 14),
 ('$61', 13),
 ('$95,000', 13),
 ('$60', 12),
 ('$50', 12),
 ('$76', 12),
 ('$54', 11),
 ('$79', 11),
 ('$98k', 11),
 ('$81', 11),
 ('$58', 10),
 ('$96', 10),
 ('$80', 10),
 ('$88', 10),
 ('$78', 9),
 ('$91', 9),
 ('$68', 9),
 ('$84', 9),
 ('$87', 9),
 ('$94', 9),
 ('$73', 8),
 ('$72', 8),
 ('$75,000', 8),
 ('$104k', 8),
 ('$110,000', 8),
 ('$100,000', 8),
 ('$120,000', 8),
 ('$35', 7),
 ('$89', 7),
 ('$71', 7),
 ('$82', 7),
 ('$66', 7),
 ('$103k', 7),
 ('$57', 6),
 ('$62', 6),
 ('$65', 6),
 ('$83', 6),
 ('$150,000', 6),
 ('$63', 6),
 ('$101k', 6),
 ('$99', 6),
 ('$95', 5),
 ('$70,000', 5),
 ('$70', 5),
 ('$89,834', 5),
 ('$116,788', 5),
 ('$90k', 5),
 ('$64', 5),
 ('$85', 5),
 ('$108k', 5),
 ('$86', 5),
 ('$114k', 5),
 ('$140,000', 5),
 ('$74', 5),
 ('$55', 5),
 ('$115,000', 4),
 ('$30', 4),
 ('$36', 4),
 ('$25', 4),
 ('$27', 4),
 ('$75', 4),
 ('$3b', 4),
 ('$130,000', 4),